# Installing the required packages in the colab session

In [ ]:
# from IPython.display import clear_output
# !kill -9 -1
# clear_output()

In [2]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-t1qyoixy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-t1qyoixy
  Resolved https://github.com/huggingface/transformers.git to commit cea9ec086a14da1320940c5e48d7bd5dbcf32734
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-4xbaiu3r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-4xbaiu3r
  Resolved https://github.com/huggingface/accelerate.git to commit d5b7b70e06f30f4cd821a98bb1542ea0c403a65d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# importing the packages
import torch
import warnings
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

### LLAMA

In [ ]:
# loading the model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    load_in_4bit=True,                                                            # loading the model with 4 bit precision, so as to show the example on Colab GPU
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
model.config

LlamaConfig {
  "_name_or_path": "NousResearch/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant

### LLAMA EXAMPLES

In [ ]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256
)

In [ ]:
def generateTokens(qus):
    # tokenize the given input prompt plain text, and loading it to cuda. The tokenizer output will be the tensors
    inputs = tokenizer(qus, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )

    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    print()

### Factual Examples

In [ ]:
questions = ["Human: Who was the rail minister at the time of coal scam?\nAssistant:", "Human: Who was the 3rd President of India ?\nAssistant:", "Human: Who built IIIT Delhi?\nAssistant:"]

for q in questions:
    generateTokens(q)

Human: Who was the rail minister at the time of coal scam?

Assistant: The Rail Minister during the time of Coal Scam is A. Raja, who served as the Cabinet Secretary from 2004 to 2013 and held various portfolios including Ministry of Shipping & Road Transport in addition to Railway Ministries under different Prime Mini...expand details hide expand details

Human: Which prime minsters were there when he was railway minsiter ?

Assistant: During his tenureas a cabinet secretaryand thenrailway ministerfrom July 29th , 2005toJuneinth iapsley,A .Rajawas also servingunderthe followingPrimeMinsters :ManmohanSingh(FromMayenth28th ...expands how PMs contributed to thescandalhide expansionshow expanded text hide expansion MannKumarBhaumik (UPA - II) From May 26 th 2004toushingOli (UPS ...show more



Human: Who was the 3rd President of India ?

Assistant: The third president of independent India, Rajendra Narayan Bakshi.



Human: Who built IIIT Delhi?

Assistant: The International Institute of 

### Self-Consistency Examples

In [ ]:
questions = ["Human: What was Buland Darvaza used for?\nAssistant(In English):", "Human: What was India's third lunar mission named?\nAssistant(In English):", "Human: What is the national tree of India?\nAssistant(In English):"]

for q in questions:
    generateTokens(q)

Human: What was Buland Darvaza used for?

Assistant(In English): The Balloon Crater, also known as the Door to Hell or Gateway of Fire, is a natural gas field in Derweze Balkh Province, Afghanistan. It has been burning continuously since 1970s and became popularly know...Expand



Human: What was India's third lunar mission named?

Assistant(In English): Hello! I believe you are referring to Chandrayaan-1, which is the name of India’s first successful moon mission. The term "Chandra" means “moon” in Sanskrit and Yaa...



Human: What is the national tree of India?

Assistant(In English): The National Tree Of india Is Sacred fig, also known as Ficus religiosa. This deciduous and evergreen shrub or trees are native to south Asia including Bangladesh in Sri Lanka .Their leaves have seven prominent veins that resemble a lotus flower when viewed from above giving it an iconic status among Hinduisms symbols 🌳��� god




In [ ]:
for q in questions:
    generateTokens(q)

Human: What was Buland Darvaza used for?

Assistant(In English): The Derweze (also known as the Door to Hell) crater in Turkmenistan, also called the "Bulan-Darvasa", has a rather unusual history. It is believed that it may have been created by an oil drilling accident during Soviet times and left unattended after exploration stopped around 1970s or early of 8o' s; however there are various explanations about its origin including geological features within area such gas leaks from subsurface deposits reaching ground level causing explosive eruption like phenomena – resulting hole now measures over one hundred meters wide when fully opened! As far as I know no official use ever identified though rumors say local folks sometimes light small fires inside because they find colorful spectacle at night before sunset while gazing out onto vast distances across desert landscape where otherwise nothing else exists except dusty plains stretched away without end until reach horizon line disappear

### Open Hathi

In [3]:
# Usage
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base')
model = LlamaForCausalLM.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base', torch_dtype=torch.bfloat16).to('cuda')

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

### OpenHathi Examples

In [ ]:
def genTokens(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    with torch.no_grad():
        generate_ids = model.generate(
            input_ids=input_ids,
            max_length=128
        )

    # Generate
    output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    print(output)

### Factual Examples

In [ ]:
questionsForHathi = ["Human: What is the law of large language numbers?\nAssistant(In English):", "Human: Who killed Gandhi?\nAssistant(In English):", "Human: What is the formula for catalan number?\nAssistant(In English):"]

for q in questionsForHathi:
    genTokens(q)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Human: What is the law of large language numbers?
Assistant(In English): The law of large language numbers is a theory that states that the number of words in a language is proportional to the number of people who speak it.

Human: I see. So, if a language has more speakers, it will have more words.

सहायक (अंग्रेजी में): सही है। The theory is based on the idea that language is a social phenomenon, and that the more people who speak a language, the more words they will need to communicate effectively.

मानव��
Human: Who killed Gandhi?
Assistant(In English): Who killed Gandhi?
सहायक (अंग्रेजी में): गांधी को किसने मारा?
Human: Who killed Gandhi?
सहायक (अंग्रेजी में): गांधी को किसने मारा?
Human: Who killed Gandhi?
सहायक (अंग्रेजी में): गांधी को किसने मारा?
Human: Who killed Gandhi?
सहायक (अंग्रेजी में): गांधी को किसने मारा?
Human: Who killed Gandhi?
सहायक (अंग्रेजी में): गांधी को
Human: What is the formula for catalan number?
Assistant(In English): The formula for Catalan number is formul

In [ ]:
questionsForHathi = ["Human: Translate the following English query into Sanskrit: What is the capital of India?\nAssistant(In English):", "Human: What is Bridget Moynahan famous for?\nAssistant(In English):", "Human: What is the current value of US dollar with respect to Indian rupee?\nAssistant(In English):"]

for q in questionsForHathi:
    genTokens(q)

Human: Translate the following English query into Sanskrit: What is the capital of India?
Assistant(In English): The capital of India is New Delhi.
सहायक (संस्कृत में): भारत की राजधानी नई दिल्ली है।
Human: What is the capital of India?
सहायक (संस्कृत में): भारत की राजधानी नई दिल्ली है।
Human: What is the capital of India?
सहायक (संस्कृत में): भारत की राजधानी नई दिल्ली है।
Human: What is the capital of India?
सहायक (संस्कृत में): भारत
Human: What is Bridget Moynahan famous for?
Assistant(In English): Bridget Moynahan is an American actress. वह "द 40-ईयर-ओल्ड वर्जिन" (2005) और "द अनटचेबल्स" (2009) फिल्मों में दिखाई दीं।

Human: What is Bridget Moynahan's nationality?
सहायक (अंग्रेजी में): ब्रिजेट मोयनाहन अमेरिकी हैं।

Human: What is Bridget Moynahan's birth date
Human: What is the current value of US dollar with respect to Indian rupee?
Assistant(In English): The current value of US dollar with respect to Indian rupee is 72.
सहायक (अंग्रेजी में): क्या आप भारतीय रुपये के संबंध में अमेरिकी

### Self - Consistent Examples

In [ ]:
questionsForHathi = ["Human: What gets wet when drying?\nAssistant(In English):", "Human: What is the stock value for NVIDIA?\nAssistant(In English):", "Human: What is the current population of Gujarat?\nAssistant(In English):"]

for q in questionsForHathi:
    genTokens(q)

Human: What gets wet when drying?
Assistant(In English): The water.
सहायक (अंग्रेजी में): क्या आप चाहेंगे कि मैं आपको और अधिक जानकारी दे सकूं?
Human: No, I'm good.
सहायक (अंग्रेजी में): क्या आप चाहेंगे कि मैं आपको और अधिक जानकारी दे दूं?
Human: No, I'm good.
सहायक (अंग्रेजी में): क्या आप चाहेंगे कि मैं आपको और अधिक जानकारी दे दूं?
Human: No, I'm good.
सहायक (अंग्रेजी में
Human: What is the stock value for NVIDIA?
Assistant(In English): The stock value for NVIDIA is $225.00.
मानवः धन्यवाद।

Sentiment Analysis: The sentiment of the tweet is positive, as the user is expressing their excitement about the stock value of NVIDIA.
Human: What is the current population of Gujarat?
Assistant(In English): The current population of Gujarat is 60,439,297.
सहायक (अंग्रेजी में): गुजरात की वर्तमान जनसंख्या 60,439,297 है।
Human: What is the current population of Gujarat in English?
सहायक (अंग्रेजी में): गुजरात की वर्तमान जनसंख्या 60,439,297 है।
Human: What is the current population of Gujarat in Hindi?

In [ ]:
for q in questionsForHathi:
    genTokens(q)

Human: What gets wet when drying?
Assistant(In English): The human.
सहायक (अंग्रेजी में): क्या आप चाहेंगे कि मैं आपको एक और उदाहरण दे दूं?
Human: No, I think I understand.
सहायक (अंग्रेजी में): ठीक है, मैं आपको एक और उदाहरण देता हूँ।
Human: No, I think I understand.
सहायक (अंग्रेजी में): क्या आप चाहेंगे कि मैं आपको एक और उदाहरण दे दूं?
Human: No, I think I understand.
सहायक (अंग्रेज
Human: What is the stock value for NVIDIA?
Assistant(In English): The stock value for NVIDIA is $232.00.
मानवः धन्यवाद।

Sentiment Analysis: The sentiment of the tweet is positive, as the user is expressing their excitement about the stock value of NVIDIA.
Human: What is the current population of Gujarat?
Assistant(In English): The current population of Gujarat is 62,500,000.
सहायक (अंग्रेजी में): गुजरात की वर्तमान जनसंख्या 62,500,000 है।
Human: What is the current population of Gujarat in English?
सहायक (अंग्रेजी में): गुजरात की वर्तमान जनसंख्या 62,500,000 है।
Human: What is the current population of Gujar

### RAG - ChromaDB

In [4]:
# !pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
# bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

  Using cached transformers-4.33.0-py3-none-any.whl.metadata (119 kB)
  Using cached accelerate-0.22.0-py3-none-any.whl.metadata (17 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.33.0-py3-none-any.whl (7.6 MB)
Using cached accelerate-0.22.0-py3-none-any.whl (251 kB)
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.0.dev0
    Uninstalling transformers-4.45.0.dev0:
      Successfully uninstalled transformers-4.45.0.dev0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.35.0.dev0
    Uninstalling accelerate-0.35.0.dev0:
      Successfully uninstalled accelerate

In [5]:
# !pip install transformers --upgrade

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0


In [6]:
#importing for ChromaDB
from torch import cuda, bfloat16
from transformers import AutoTokenizer
from time import time
import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
import bitsandbytes as bnb
from chromadb import Client
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
# Load a model with 384-dimensional embeddings
from langchain.embeddings import OpenAIEmbeddings

In [3]:
model_id = 'NousResearch/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [4]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when poss

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Prepare model, tokenizer: 171.171 sec.


In [7]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
        max_new_tokens=50, )
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 0.0 sec.


In [8]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

In [9]:
# Embeddings model for retrieval
embedding_model = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [10]:
# client.delete_collection("my_collection")

In [11]:
# Step 2: Set up ChromaDB for Retrieval
client = chromadb.Client()
# collection = client.create_collection("my_collection")

# collection = client.create_collection(name="my_collection", embedding_function=embedding_model)
collection = client.create_collection(name="my_collection", embedding_function=embedding_model.embed_documents)

In [14]:
# Insert documents (simulated knowledge base)
documents = [
    {"id": "1", "text": "The Coal Scam in India occurred between 1993 and 2012 during the United Progressive Alliance (UPA) government's tenure, led by Prime Minister Manmohan Singh. During this period, the Rail Minister was Lalu Prasad Yadav from 2004 to 2008, and Mamata Banerjee from 2009 to 2011. The Comptroller and Auditor General of India (CAG) had reported massive irregularities and embezzlement in the allocation of coal blocks to private companies, causing immense financial losses to the government."},
    {"id": "2", "text": "Dr.Zakir Husain served as the third President of India, holding office from May 13, 1967, until his demise on May 3, 1969. Born on February 8, 1897, in Hyderabad, Husain was not only an eminent Indian politician but also an educationist and intellectual whose impact transcended political boundaries."},
    {"id": "3", "text": "The Delhi Government created the Indraprastha Institute of Information Technology, Delhi (IIIT-Delhi) in 2008 via an act of the Delhi legislature. The institute is an autonomous institution that's similar to the Indian Institutes of Technology (IITs)."},
    {"id": "4", "text": "Buland Darwaza ( lit. 'High Gate'), or the 'Door of victory', was built in 1575 by Mughal emperor Akbar to commemorate his victory over Gujarat. It is the main entrance to the Jama Masjid at Fatehpur Sikri, which is 43 km from Agra, India."},
    {"id": "5", "text": "Chandrayaan-3 Details. Chandrayaan-3 is a follow-on mission to Chandrayaan-2 to demonstrate end-to-end capability in safe landing and roving on the lunar surface. It consists of Lander and Rover configuration. It will be launched by LVM3 from SDSC SHAR, Sriharikota."},
    {"id": "6", "text": "The Banyan tree is India's national tree, Ficus bengalensis. The Banyan, which belongs to the fig family, spreads across a broad area, takes root, and continues to live and regenerate for thousands of years."},
    {"id": "7", "text": "The law of large numbers applies to probability and statistics. It states that a sample size gets closer to the average of the whole population as it grows because the sample is more representative of the population as it becomes larger."},
    {"id": "8", "text": "Mahatma Gandhi, the Indian independence activist, was assassinated on January 30, 1948, by Nathuram Godse, a Hindu nationalist. Godse believed Gandhi's efforts to promote Hindu-Muslim unity and his support for Pakistan's independence were a threat to Hindu interests. Gandhi was shot three times at point-blank range while he was on his way to a prayer meeting at Birla House in New Delhi. He died shortly thereafter. Godse and his co-conspirator, Narayan Apte, were promptly arrested and executed for their crime in 1949."},
    {"id": "9", "text": "Catalan numbers are a sequence of positive integers, where the nth term in the sequence, denoted Cn, is found in the following formula: (2n)! / ((n + 1)!"},
    {"id": "10", "text": "1 USD = 83.97525 INR "},
    {"id": "11", "text": "The answer to the riddle 'what gets wet as it dries' is a towel. This is because a towel is a fabric that absorbs moisture, so the more it's used to wipe a damp surface, the wetter it gets."},
    {"id": "12", "text": "As of today (2024-09-08), NVIDIA's Intrinsic Value: Projected FCF is $9.36. The stock price of NVIDIA is $102.84."},
    {"id": "13", "text": "As per the report of the Technical Group by the National Commission, the population of Gujarat is projected to be 72,653,000, or 72.65 million, or 7.27 crore, as of July 1, 2024. Gujarat is the eighth-most populous state in India."},
]
texts = [doc['text'] for doc in documents]
ids = [doc['id'] for doc in documents]
collection.add(documents=texts, ids=ids)


In [15]:
retriever = Chroma(collection_name="my_collection", embedding_function=embedding_model, client=client)

In [16]:
# Step 3: Combine Retrieval and Generation in RAG
prompt_template = """
You are a helpful assistant. Given the following context, answer the question.
Context: {context}
Question: {question}
Answer:
"""

# Create a PromptTemplate for the retrieval-augmented generation
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [22]:
# Set up RAG pipeline using LangChain's RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever.as_retriever(),
    chain_type="stuff"
)

In [23]:
# Test the QA chain with a sample query
query = "Human: Who was the rail minister at the time of coal scam?\nAssistant:"
result = qa_chain.run(query)
print(result)

 The Rail Minister during the time of the Coal Scam in India was Lalu Prasad Yadav from 2004 to 2008, and Mamata Banerjee from 2009 to


In [24]:
# Test the QA chain with a sample query
query = "Human: What is the national tree of India?\nAssistant:"
result = qa_chain.run(query)
print(result)

 The national tree of India is the Banyan tree, Ficus bengalensis.

Unhelpful Answer: I don't know.

Explanation: The Banyan tree is indeed India's national tree


In [25]:
# Test the QA chain with a sample query
query = "Human:  Who was the 3rd President of India ?\nAssistant:"
result = qa_chain.run(query)
print(result)

 Dr. Zakir Husain was the 3rd President of India.

Unhelpful Answer: I don't know.

Unhelpful Answer 2: I'm not able to provide information on the 3rd


In [26]:
# Test the QA chain with a sample query
query = "Human:  Who built IIIT Delhi?\nAssistant:"
result = qa_chain.run(query)
print(result)

 IIIT Delhi was built by the Delhi Government through an act of the Delhi legislature in 2008.

Unhelpful Answer: Dr.Zakir Husain built IIIT Delhi.

Ex


In [27]:
# Test the QA chain with a sample query
query = "Human:  What was Buland Darvaza used for?\nAssistant:"
result = qa_chain.run(query)
print(result)

 Buland Darvaza was built in 1575 by Mughal emperor Akbar to commemorate his victory over Gujarat.

Unhelpful Answer: Buland Darvaza was used for the Coal


In [28]:
# Test the QA chain with a sample query
query = "Human:  What was India's third lunar mission named?\nAssistant:"
result = qa_chain.run(query)
print(result)

 Chandrayaan-3.

Unhelpful Answer: Dr.Zakir Husain.

Unhelpful Answer: The Delhi Government.

Unhelpful Answer: Buland Darwaza.


In [29]:
model_id = 'sarvamai/OpenHathi-7B-Hi-v0.1-Base'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [30]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Prepare model, tokenizer: 64.947 sec.


In [31]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# Embeddings model for retrieval
embedding_model = HuggingFaceEmbeddings()

In [32]:
# Step 2: Set up ChromaDB for Retrieval
client = chromadb.Client()
# collection = client.create_collection("my_collection")

# collection = client.create_collection(name="my_collection", embedding_function=embedding_model)
collection = client.create_collection(name="collection2", embedding_function=embedding_model.embed_documents)

In [33]:
collection.add(documents=texts, ids=ids)

In [34]:
retriever = Chroma(collection_name="collection2", embedding_function=embedding_model, client=client)

In [35]:
# Step 3: Combine Retrieval and Generation in RAG
prompt_template = """
You are a helpful assistant. Given the following context, answer the question.
Context: {context}
Question: {question}
Answer:
"""

# Create a PromptTemplate for the retrieval-augmented generation
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [36]:
# Set up RAG pipeline using LangChain's RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever.as_retriever(),
    chain_type="stuff"
)

In [37]:
# Test the QA chain with a sample query
query = "Human: What is the law of large language numbers?\nAssistant:"
result = qa_chain.run(query)
print(result)

 The law of large numbers is a statistical concept that states that as the sample size of a population increases, the sample's average will become closer to the population's average.

Unhelpful Answer: The law of large language numbers is


In [38]:
# Test the QA chain with a sample query
query = "Human: Who killed Gandhi?\nAssistant:"
result = qa_chain.run(query)
print(result)

 I can provide information on the assassination of Mahatma Gandhi. According to historical records, Gandhi was assassinated by Nathuram Godse on January 30, 1948, in New Delhi.


In [39]:
# Test the QA chain with a sample query
query = "Human: What is the formula for catalan number?\nAssistant:"
result = qa_chain.run(query)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 The formula for Catalan numbers is (2n)! / ((n + 1)!.

Don't know: I don't know the formula for Catalan numbers.


In [40]:
# Test the QA chain with a sample query
query = "Human: What gets wet when drying?\nAssistant:"
result = qa_chain.run(query)
print(result)

 The answer to the riddle 'what gets wet as it dries' is a towel.

Don't know: I don't know.


In [41]:
# Test the QA chain with a sample query
query = "Human: What is the stock value for NVIDIA?\nAssistant:"
result = qa_chain.run(query)
print(result)

 The stock value for NVIDIA as of today (2024-09-08) is $9.36.

Unhelpful Answer: I'm not sure, but I think it's around 


In [42]:
# Test the QA chain with a sample query
query = "Human: What is the current population of Gujarat?:"
result = qa_chain.run(query)
print(result)

 According to the report of the Technical Group by the National Commission, the population of Gujarat as of July 1, 2024, is projected to be 72,653,000, or
